# Lab 4: Model Context Protocol (MCP) as Tools

Welcome to Lab 4 of our Strands Agent course! So far, we've created basic agents and connected them with different model providers.  Now we will empower these agents with tools that will allow them to interact with external systems, access data, and manipulate their environment. And you're going to learn how to use the [Model Context Protocol (MCP)](https://modelcontextprotocol.io/) as one of those tools.



### Let's create a Weather Agent

Strands agents offer an [Example Tools Package](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/tools_overview/#adding-tools-to-agents). You can also create [custom tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/) similar to MCP implementation, but without the need to run a dedicated server.

To create the weather agent, we use the [http_request tool](https://github.com/strands-agents/tools/blob/main/src/strands_tools/http_request.py). This tool connects Strands agents with external web services and APIs, bridging conversational AI with data sources. The tool supports multiple HTTP methods (GET, POST, PUT, DELETE), handles URL encoding and response parsing, and returns structured data from web sources.

![image.png](image/weather_agent.png)

In [ ]:

# % pip install -r requirements.txt

In [ ]:
# api_key="your-api-key"

In [2]:
from strands import Agent
from strands_tools import http_request
from strands.models import BedrockModel
import os

# Define a weather-focused system prompt
WEATHER_SYSTEM_PROMPT = """You are a weather assistant with HTTP capabilities. You can:

1. Make HTTP requests to the National Weather Service API
2. Process and display weather forecast data
3. Provide weather information for locations in the United States

When retrieving weather information:
1. First get the coordinates or grid information using https://api.weather.gov/points/{latitude},{longitude} or https://api.weather.gov/points/{zipcode}
2. Then use the returned forecast URL to get the actual forecast

When displaying responses:
- Format weather data in a human-readable way
- Highlight important information like temperature, precipitation, and alerts
- Handle errors appropriately
- Convert technical terms to user-friendly language

Always explain the weather conditions clearly and provide context for the forecast.
"""



# Configure specific Bedrock model
model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",  # Specify the model
    temperature=0.7,                         # Control randomness
    top_p=0.8,
    max_tokens=1028,                         # Control token selection
)

# Create an agent with HTTP capabilities
weather_agent = Agent(
    model = model,
    system_prompt=WEATHER_SYSTEM_PROMPT,
    tools=[http_request],  # Explicitly enable http_request tool
)

### ✅ Test the Weather agent

In [3]:
response = weather_agent("What's the weather like in Seattle?")


I'll help you check the weather in Seattle, WA. Let me get that information from the National Weather Service API.

First, I'll get the coordinates for Seattle and then retrieve the forecast.
Tool #1: http_request


╭──────────────────────────── 🚀 HTTP Request Preview: GET /points/47.6062,-122.3321 ─────────────────────────────╮
│                                                                                                                 │
│   Method    GET                                                                                                 │
│   URL       https://api.weather.gov/points/47.6062,-122.3321                                                    │
│   Headers   {}                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Sending request...

╭─────────────────────────────────────────── ✅ HTTP Response: 200 OK ────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│     Status         200 OK                                                                                       │
│     URL            https://api.weather.gov/points/47.6062,-122.3321                                             │
│     Content-Type   application/geo+json                                                                         │
│     Size           3,069 bytes (3.0 KB)                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                    Response Headers                                     
╭───────────────────────────────┬───────────────────────────────────────────────────────╮
│ Header                        │ Value                                                 │
├───────────────────────────────┼───────────────────────────────────────────────────────┤
│ Server                        │ nginx/1.20.1                                          │
│ Content-Type                  │ application/geo+json                                  │
│ Access-Control-Allow-Origin   │ *                                                     │
│ Access-Control-Expose-Headers │ X-Correlation-Id, X-Request-Id, X-Server-Id           │
│ X-Request-ID                  │ 2393d774-794b-4d07-860f-8aac2844ba42                  │
│ X-Correlation-ID              │ 274386a7                                              │
│ X-Server-ID                   │ vm-bldr-nids-apiapp15.ncep.noaa.gov                   │
│ Content-Encoding              │ gzip                                                  │
│ Content-Length                │ 807                                                   │
│ Cache-Control                 │ public, max-age=19100, s-maxage=120                   │
│ Expires                       │ Wed, 17 Sep 2025 04:05:19 GMT                         │
│ Date                          │ Tue, 16 Sep 2025 22:46:59 GMT                         │
│ Connection                    │ keep-alive                                            │
│ Vary                          │ Accept-Encoding, Accept,Feature-Flags,Accept-Language │
│ X-Edge-Request-ID             │ 1727834f                                              │
│ Strict-Transport-Security     │ max-age=31536000 ; includeSubDomains ; preload        │
╰───────────────────────────────┴───────────────────────────────────────────────────────╯

...

╭───────────────────────── 🚀 HTTP Request Preview: GET /gridpoints/SEW/125,68/forecast ──────────────────────────╮
│                                                                                                                 │
│   Method    GET                                                                                                 │
│   URL       https://api.weather.gov/gridpoints/SEW/125,68/forecast                                              │
│   Headers   {}                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Sending request...

╭─────────────────────────────────────────── ✅ HTTP Response: 200 OK ────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│     Status         200 OK                                                                                       │
│     URL            https://api.weather.gov/gridpoints/SEW/125,68/forecast                                       │
│     Content-Type   application/geo+json                                                                         │
│     Size           12,890 bytes (12.6 KB)                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                    Response Headers                                     
╭───────────────────────────────┬───────────────────────────────────────────────────────╮
│ Header                        │ Value                                                 │
├───────────────────────────────┼───────────────────────────────────────────────────────┤
│ Server                        │ nginx/1.20.1                                          │
│ Content-Type                  │ application/geo+json                                  │
│ Last-Modified                 │ Tue, 16 Sep 2025 21:26:55 GMT                         │
│ Access-Control-Allow-Origin   │ *                                                     │
│ Access-Control-Expose-Headers │ X-Correlation-Id, X-Request-Id, X-Server-Id           │
│ X-Request-ID                  │ bb5a01a7-0886-4d2d-8465-c99af6e550d2                  │
│ X-Correlation-ID              │ 35fa65ae                                              │
│ X-Server-ID                   │ vm-bldr-nids-apiapp5.ncep.noaa.gov                    │
│ Content-Encoding              │ gzip                                                  │
│ Content-Length                │ 1476                                                  │
│ Cache-Control                 │ public, max-age=900, s-maxage=3600                    │
│ Expires                       │ Tue, 16 Sep 2025 23:02:02 GMT                         │
│ Date                          │ Tue, 16 Sep 2025 22:47:02 GMT                         │
│ Connection                    │ keep-alive                                            │
│ Vary                          │ Accept-Encoding, Accept,Feature-Flags,Accept-Language │
│ X-Edge-Request-ID             │ 1727842c                                              │
│ Strict-Transport-Security     │ max-age=31536000 ; includeSubDomains ; preload        │
╰───────────────────────────────┴───────────────────────────────────────────────────────╯

Here's the current weather forecast for Seattle:

Right now (This Afternoon):
- Temperature: 87°F
- Conditions: Sunny
- Wind: Southeast at 8 mph
- Precipitation chance: 0%

Tonight:
- Low temperature: 59°F (rising to 61°F overnight)
- Conditions: Clear
- Wind: South southeast at 6 mph with gusts up to 22 mph
- Precipitation chance: 1%

Tomorrow (Wednesday):
- High temperature: 76°F
- Conditions: Sunny
- Wind: West 3 to 9 mph
- Precipitation chance: 2%

Extended Forecast Highlights:
- The weather will remain mostly clear and pleasant through Friday
- Temperatures will stay moderate, with highs in the 70s and lows in the 50s
- There's a chance of rain starting Saturday (52% chance) with more likely rain on Saturday night (75% chance)
- The weekend looks somewhat wet with rain likely on Sunday

Overall, it's currently a warm, sunny day in Seattle, with pleasant conditions continuing through the end of the work week before some rain moves in for the weekend.

In [4]:
print(response.message['content'][0]['text'])

Here's the current weather forecast for Seattle:

Right now (This Afternoon):
- Temperature: 87°F
- Conditions: Sunny
- Wind: Southeast at 8 mph
- Precipitation chance: 0%

Tonight:
- Low temperature: 59°F (rising to 61°F overnight)
- Conditions: Clear
- Wind: South southeast at 6 mph with gusts up to 22 mph
- Precipitation chance: 1%

Tomorrow (Wednesday):
- High temperature: 76°F
- Conditions: Sunny
- Wind: West 3 to 9 mph
- Precipitation chance: 2%

Extended Forecast Highlights:
- The weather will remain mostly clear and pleasant through Friday
- Temperatures will stay moderate, with highs in the 70s and lows in the 50s
- There's a chance of rain starting Saturday (52% chance) with more likely rain on Saturday night (75% chance)
- The weekend looks somewhat wet with rain likely on Sunday

Overall, it's currently a warm, sunny day in Seattle, with pleasant conditions continuing through the end of the work week before some rain moves in for the weekend.


### ✅  Weather agent with a custom tool (Python Tool)

![image.png](image/weather_agent_tool.png)

In [8]:
from strands import Agent
from strands_tools import http_request
from strands.tools import tool
from strands.models import BedrockModel
from typing import Dict, Any

# Define a function to convert Fahrenheit to Celsius
@tool
def fahrenheit_to_celsius(params: Dict[str, Any]) -> Dict[str, Any]:
    """
    Convert temperature from Fahrenheit to Celsius.

    Args:
        params: A dictionary containing:
            - fahrenheit: The temperature in Fahrenheit to convert

    Returns:
        A dictionary containing:
            - celsius: The temperature in Celsius
            - original_fahrenheit: The original Fahrenheit value
    """
    try:
        fahrenheit = float(params.get("fahrenheit"))
        celsius = (fahrenheit - 32) * 5/9
        return {
            "celsius": round(celsius, 2),
            "original_fahrenheit": fahrenheit
        }
    except (TypeError, ValueError):
        return {
            "error": "Invalid input. Please provide a valid number for fahrenheit."
        }


# Define a weather-focused system prompt
WEATHER_SYSTEM_PROMPT_C_TOOL = """You are a weather assistant with HTTP capabilities. You can:

1. Make HTTP requests to the National Weather Service API
2. Process and display weather forecast data
3. Provide weather information for locations in the United States
4. Use fahrenheit_to_celsius to convert temperatures between Fahrenheit and Celsius

When retrieving weather information:
1. First get the coordinates or grid information using https://api.weather.gov/points/{latitude},{longitude} or https://api.weather.gov/points/{zipcode}
2. Then use the returned forecast URL to get the actual forecast

When displaying responses:
- Format weather data in a human-readable way
- Highlight important information like temperature, precipitation, and alerts
- Handle errors appropriately
- Convert technical terms to user-friendly language

Always explain the weather conditions clearly and provide context for the forecast.
"""

model = BedrockModel(
    model_id="anthropic.claude-3-5-sonnet-20241022-v2:0",  # Specify the model
    temperature=0.7,                         # Control randomness
    top_p=0.8,
    max_tokens=1028,                         # Control token selection
)

# Create an agent with HTTP capabilities
weather_agent_c_tool = Agent(
    model = model,
    system_prompt=WEATHER_SYSTEM_PROMPT_C_TOOL,
    tools=[http_request,fahrenheit_to_celsius],  # Explicitly enable http_request tool
)

In [6]:
print(weather_agent_c_tool.tool_names)

['http_request', 'fahrenheit_to_celsius']


In [9]:
response_c_tool = weather_agent_c_tool("What is the weather like in Seattle? Please give me the answer in degrees Celsius")


I'll help you get the weather information for Seattle, WA. I'll need to make two API calls - first to get the grid points for Seattle, and then to get the actual forecast. After that, I'll convert the temperatures to Celsius for you.

First, let's get the coordinates for Seattle:
Tool #1: http_request


╭──────────────────────────── 🚀 HTTP Request Preview: GET /points/47.6062,-122.3321 ─────────────────────────────╮
│                                                                                                                 │
│   Method    GET                                                                                                 │
│   URL       https://api.weather.gov/points/47.6062,-122.3321                                                    │
│   Headers   {}                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Sending request...

╭─────────────────────────────────────────── ✅ HTTP Response: 200 OK ────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│     Status         200 OK                                                                                       │
│     URL            https://api.weather.gov/points/47.6062,-122.3321                                             │
│     Content-Type   application/geo+json                                                                         │
│     Size           3,069 bytes (3.0 KB)                                                                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                    Response Headers                                     
╭───────────────────────────────┬───────────────────────────────────────────────────────╮
│ Header                        │ Value                                                 │
├───────────────────────────────┼───────────────────────────────────────────────────────┤
│ Server                        │ nginx/1.20.1                                          │
│ Content-Type                  │ application/geo+json                                  │
│ Access-Control-Allow-Origin   │ *                                                     │
│ Access-Control-Expose-Headers │ X-Correlation-Id, X-Request-Id, X-Server-Id           │
│ X-Request-ID                  │ b7d80910-7e14-4259-b511-7fc992622387                  │
│ X-Correlation-ID              │ 58217cc                                               │
│ X-Server-ID                   │ vm-bldr-nids-apiapp13.ncep.noaa.gov                   │
│ Content-Encoding              │ gzip                                                  │
│ Content-Length                │ 807                                                   │
│ Cache-Control                 │ public, max-age=57913, s-maxage=120                   │
│ Expires                       │ Wed, 17 Sep 2025 15:01:53 GMT                         │
│ Date                          │ Tue, 16 Sep 2025 22:56:40 GMT                         │
│ Connection                    │ keep-alive                                            │
│ Vary                          │ Accept-Encoding, Accept,Feature-Flags,Accept-Language │
│ X-Edge-Request-ID             │ 27073c1                                               │
│ Strict-Transport-Security     │ max-age=31536000 ; includeSubDomains ; preload        │
╰───────────────────────────────┴───────────────────────────────────────────────────────╯

...

╭───────────────────────── 🚀 HTTP Request Preview: GET /gridpoints/SEW/125,68/forecast ──────────────────────────╮
│                                                                                                                 │
│   Method    GET                                                                                                 │
│   URL       https://api.weather.gov/gridpoints/SEW/125,68/forecast                                              │
│   Headers   {}                                                                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Sending request...

╭─────────────────────────────────────────── ✅ HTTP Response: 200 OK ────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│     Status         200 OK                                                                                       │
│     URL            https://api.weather.gov/gridpoints/SEW/125,68/forecast                                       │
│     Content-Type   application/geo+json                                                                         │
│     Size           12,890 bytes (12.6 KB)                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

                                    Response Headers                                     
╭───────────────────────────────┬───────────────────────────────────────────────────────╮
│ Header                        │ Value                                                 │
├───────────────────────────────┼───────────────────────────────────────────────────────┤
│ Server                        │ nginx/1.20.1                                          │
│ Content-Type                  │ application/geo+json                                  │
│ Last-Modified                 │ Tue, 16 Sep 2025 21:26:55 GMT                         │
│ Access-Control-Allow-Origin   │ *                                                     │
│ Access-Control-Expose-Headers │ X-Correlation-Id, X-Request-Id, X-Server-Id           │
│ X-Request-ID                  │ e1aa0042-4953-481e-b09b-9f3fb719eea6                  │
│ X-Correlation-ID              │ 14110cb8                                              │
│ X-Server-ID                   │ vm-bldr-nids-apiapp14.ncep.noaa.gov                   │
│ Content-Encoding              │ gzip                                                  │
│ Content-Length                │ 1476                                                  │
│ Cache-Control                 │ public, max-age=436, s-maxage=3600                    │
│ Expires                       │ Tue, 16 Sep 2025 23:03:58 GMT                         │
│ Date                          │ Tue, 16 Sep 2025 22:56:42 GMT                         │
│ Connection                    │ keep-alive                                            │
│ Vary                          │ Accept-Encoding, Accept,Feature-Flags,Accept-Language │
│ X-Edge-Request-ID             │ 2707457                                               │
│ Strict-Transport-Security     │ max-age=31536000 ; includeSubDomains ; preload        │
╰───────────────────────────────┴───────────────────────────────────────────────────────╯

Let me convert the current temperature from Fahrenheit to Celsius:
Tool #3: fahrenheit_to_celsius
Here's the current weather in Seattle:

Current Conditions:
- Temperature: 30.6°C (87°F)
- Conditions: Sunny
- Wind: Southeast at 8 mph
- Precipitation Chance: 0%

Tonight's Forecast:
- Low: 15°C (59°F)
- Conditions: Clear
- Wind: South southeast at 6 mph

Tomorrow's Forecast:
- High: 24.4°C (76°F)
- Conditions: Sunny
- Wind: West 3 to 9 mph

Extended Outlook:
The weather in Seattle will remain pleasant for the next few days with sunny to mostly sunny conditions. However, there's a significant change coming this weekend, with a 52% chance of rain on Saturday, increasing to 75% chance of rain Saturday night. Temperatures will gradually cool down from today's warm conditions.

In [ ]:
print(response_c_tool.message['content'][0]['text'])

Here's the current weather in Seattle:

Current Conditions:
- Temperature: 30.6°C (87°F)
- Conditions: Sunny
- Wind: Southeast at 8 mph
- Precipitation Chance: 0%

Tonight's Forecast:
- Low: 15°C (59°F)
- Conditions: Clear
- Wind: South southeast at 6 mph

Tomorrow's Forecast:
- High: 24.4°C (76°F)
- Conditions: Sunny
- Wind: West 3 to 9 mph

Extended Outlook:
The weather in Seattle will remain pleasant for the next few days with sunny to mostly sunny conditions. However, there's a significant change coming this weekend, with a 52% chance of rain on Saturday, increasing to 75% chance of rain Saturday night. Temperatures will gradually cool down from today's warm conditions.


## The Model Context Protocol (MCP)

The Model Context Protocol (MCP) is an open protocol that standardized way to expose and consume tools across different systems. 

This approach is ideal for creating reusable tool collections that can be shared across multiple agents or applications. 

You can create your own MCP server to extend agent capabilities. 

![image.png](image/mcp.png)

### MCP Architecture Components

- **MCP Client**: Integrated in Strands Agent Framework 

- **MCP Server**: Independent service hosting tools

- **MCP Tools**: Specific functionalities (calculator, weather, etc.)


## Let's create a Calculator MCP server.

In this section, we will do a walkthrough of selected code blocks, that have been used to build [mcp_calculator.py](mcp_calulator.py) to create a simple MCP server that provides limited calculator functionality.

![calculator.png](image/calculator.png)


### ✅ Create the MCP server 


```python
from mcp.server import FastMCP

mcp = FastMCP("Calculator Server")

@mcp.tool(description="Add two numbers together")
def add(x: int, y: int) -> int:
"""Add two numbers and return the result."""
    return x + y

# Define a subtraction tool
@mcp.tool(description="Subtract one number from another")
def subtract(x: int, y: int) -> int:
        """Subtract y from x and return the result.

        Args:
            x: Number to subtract from
            y: Number to subtract

        Returns:
            The difference (x - y)
        """
    return x - y

# Define a multiplication tool
@mcp.tool(description="Multiply two numbers together")
def multiply(x: int, y: int) -> int:
        """Multiply two numbers and return the result.

        Args:
            x: First number
            y: Second number

        Returns:
            The product of x and y
        """
    return x * y

# Define a division tool
@mcp.tool(description="Divide one number by another")
def divide(x: float, y: float) -> float:
        """Divide x by y and return the result.

        Args:
            x: Numerator
            y: Denominator (must not be zero)

        Returns:
            The quotient (x / y)

        Raises:
            ValueError: If y is zero
        """
    if y == 0:
        raise ValueError("Cannot divide by zero")
    return x / y
```

[mcp_calculator.py](mcp_calulator.py) uses Streamable HTTP as MCP Server Connection 

```python
mcp.run(transport="streamable-http")
```

### ✅ Connecting the server to the Strands Agent

The following code shows how to create a Strands agent, and connect it to a MCP server:

```python
from mcp.client.streamable_http import streamablehttp_client
from strands import Agent
from strands.tools.mcp.mcp_client import MCPClient
from strands.models.anthropic import AnthropicModel

model = AnthropicModel(
    client_args={
        "api_key": "<KEY>", #Here your credentials
    },
    # **model_config
    max_tokens=1028,
    model_id="claude-sonnet-4-20250514",
    params={
        "temperature": 0.7,
    }
)
    # Create a system prompt that explains the calculator capabilities
    system_prompt = """
    You are a helpful calculator assistant that can perform basic arithmetic operations.
    You have access to the following calculator tools:
    - add: Add two numbers together
    - subtract: Subtract one number from another
    - multiply: Multiply two numbers together
    - divide: Divide one number by another
    
    When asked to perform calculations, use the appropriate tool rather than calculating the result yourself.
    Explain the calculation and show the result clearly.
    """

def create_streamable_http_transport():
   return streamablehttp_client("http://localhost:8000/mcp/")

streamable_http_mcp_client = MCPClient(create_streamable_http_transport)

# Use the MCP server in a context manager
with streamable_http_mcp_client:
    # Get the tools from the MCP server
    tools = streamable_http_mcp_client.list_tools_sync()
    
    # Create an agent with the MCP tools
    agent = Agent(model=model,system_prompt=system_prompt,tools=tools)

```




### ✅ Test the calculator agent

Go to the terminal and run the app

```

python3 mcp_calulator.py

```

![calculator.gif](image/calculator.gif)


